## Titanic Linear Model

In [3]:
%%datalabframework getfilename

In [6]:
import datalabframework as dlf
logger = dlf.log.initLogger(__name__, kafka_topic="datalab", kafka_servers="kafka:9092")

2017-04-30 14:06:46,314 - jovyan - __main__ - INFO - init - {'notebook': {'filename': 'lr.ipynb', 'filepath': '/home/jovyan/work/notebooks/models'}, 'datalab': {'framework': '0.1'}, 'project': {'rootpath': '/home/jovyan/work/notebooks', 'main': 'main.ipynb'}}


In [7]:
# EXPORT

from pyspark.ml.classification import LogisticRegression

In [8]:
# EXPORT

def lr_model(df):
   
    lr = LogisticRegression(maxIter=10, regParam=0.01)
    model = lr.fit(df)

    return model

In [11]:
DATA_ROOT = '/home/jovyan/work/data'
d = {   
    'input_sample' : 1.0,
    'input_source' : DATA_ROOT + '/titanic/data/set/train/features.parquet',
    'output_model' : DATA_ROOT + '/titanic/models/lr.model'
}
p = dlf.params.config_fromdict(d)

In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(dlf.project.filename()).getOrCreate()
spark.version

'2.1.0'

In [13]:
df = spark.read.parquet(p.input_source)
df.printSchema()
df.show(10, truncate=False)

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Title: string (nullable = true)

+-----------+--------+------+------+------------------+-----+-----+-------+--------+------+
|PassengerId|Survived|Pclass|Sex   |Age               |SibSp|Parch|Fare   |Embarked|Title |
+-----------+--------+------+------+------------------+-----+-----+-------+--------+------+
|1          |0       |3     |male  |22.0              |1    |0    |7.25   |S       |Mr    |
|2          |1       |1     |female|38.0              |1    |0    |71.2833|C       |Mrs   |
|3          |1       |3     |female|26.0              |0    |0    |7.925  |S       |Miss  |
|4          |1       |1     |female|35.0              |1   

In [14]:
from features import features
df_features = features.feature_vector(df, 
                  'PassengerId', 
                  'Survived', 
                  ['Pclass', 
                   'Sex', 
                   'Age', 
                   'SibSp', 
                   'Parch',
                   'Fare', 
                   'Embarked', 
                   'Title'])

NameError: name 'InteractiveShell' is not defined

In [ ]:
model = lr_model(df_features)

In [ ]:
# no overwrite mode in python for spark yet :(
import shutil
try:
    shutil.rmtree(p.output_model)
except:
    pass

In [ ]:
model.save(p.output_model)

In [ ]:
df_features.show(n=5,truncate=False)

In [ ]:
prediction = model.transform(df_features)
prediction.show(5)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()

metric = evaluator.evaluate(prediction)
metric_name = evaluator.getMetricName()

print("Evaluation {} : {}".format(metric_name, metric))